# QUANT.PY - 003

* Lives toda terça-feira, 20h.
  
  ### Grupo das lives na descrição:
    * O código/material será enviado lá. Está na descrição do grupo. 
    * Ative o lembrete da nossa próxima live! 
<br>

* Vai rolar um evento chamado Arquivos Confidenciais semana que vem onde eu vou mostrar 3 modelos de investimento lucrativos e inéditos. No final do evento vai rolar uma promoção pra entrar no Códigopy, que é minha formação de analise de dados pro mercado financeiro. 2º link da descrição.

----

# Projeto: Comprar ações na máxima história: armadilha o oportunidade?

* Vamos supor que toda vez que uma ação bate na máxima histórica, nós compramos no fechamento e seguramos por 3 dias. Nesses 3 dias seguintes, a ação tende continuar subindo ou regredir a média? 


### Passo a passo:

   **Passo 1** - Importar bibliotecas
   
   **Passo 2** - Pegar dados de cotação do Yahoo Finance
   
   **Passo 3** - Preparar e calcular dados importantes pro modelo
   
   **Passo 4** - Gerar sinais de compra
   
   **Passo 5** - Visualizar gráfico de máximas históricas
   
   **Passo 6** - Gerar sinais de venda
   
   **Passo 7** - Criar um ID para todos os trades históricos na tabela
   
   **Passo 8** - Calcular retornos de todos os trades

   **Passo 9** - Calcular estatísticas do nosso modelo de investimento (EV, Trades certos, Rentabilidade acumulada)

   
   <br>
   
   
-------------------

# Passo 1 - Importar as bibliotecas

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk
from matplotlib.ticker import FuncFormatter
plt.style.use("cyberpunk")

# Passo 2 - Pegar dados de cotação do Yahoo Finance

In [ ]:
ticker = "KLBN11.SA"
periodo_de_hold = 3

In [ ]:
dados = yf.download(ticker)

dados["Close"].plot()

# Passo 3 - Preparar e calcular dados importantes pro modelo

In [ ]:
dados["retorno"] = dados["Close"].pct_change()

In [ ]:
dados["maxima_historica"] = dados["Close"].cummax() #maxima historica ate aquele momento
dados["maxima_historica_stop_venda"] = dados["maxima_historica"].shift(periodo_de_hold) #qual era a maxima historica 3 dias atrás. Vai ajudar no sinal de venda. 
dados["maxima_historica_sinal_compra"] = dados["maxima_historica"].shift(1) #qual era a máxima histórica ontem. Vai ajudar no sinal de compra

# Passo 4 - Gerar sinais de compra

* Nosso objetivo final quando estamos fazendo um modelo de compra ou venda baseado em sinais é criar uma coluna de 0, 1 ou -1 que indicam qual era o nosso posicionamento em um determinado período do tempo no ativo. Criando essa coluna, basta multiplar pela coluna do retorno pra chegar no retorno final do modelo!. 

In [ ]:
dados["sinal"] = 0
dados["sinal"] = (dados["maxima_historica"] != dados["maxima_historica_sinal_compra"]).astype(int)

# Passo 5 - Visualizar gráfico de máximas históricas

In [ ]:
dias_de_maxima = dados[dados["sinal"] == 1]["Close"]

In [ ]:
fig, ax = plt.subplots()

ax.scatter(dias_de_maxima.index, dias_de_maxima.values, color = "red", s=4, zorder=2)
ax.plot(dados.index, dados["Close"].values, zorder = 1)

# Passo 6 - Gerar sinais de venda

In [ ]:
dados["sinal"] = np.where(dados["sinal"] == 0, np.where(dados["maxima_historica"] != dados["maxima_historica_stop_venda"], 1, 0), dados["sinal"])

# Passo 7 - Criar um ID para todos os trades históricos na tabela

* Nosso objetivo é conseguir agrupar todos os trades através de um ID

In [ ]:
dados['trades'] = (dados['sinal'] != dados['sinal'].shift()).cumsum() 

dados.head(202) #head 50 e 202

In [ ]:
dados['trades'] = dados['trades'].where(dados['sinal'] == 1) #só interessado nos IDs quando estávamos comprados 

In [ ]:
dados = dados.dropna(subset = "trades")

dados.head(50)

# Passo 8 - Calcular retornos de todos os trades

In [ ]:
dados["retorno_cota"] = 1 + dados["retorno"]

In [ ]:
dados["rentabilidade_trades"] = dados.groupby("trades")["retorno_cota"].cumprod() - 1

In [ ]:
dados = dados.sort_index(ascending = False)
dados = dados.drop_duplicates("trades") #deixando apenas o ultimo dia pra pegar o retorno final do trade
dados = dados.sort_index(ascending = True)
dados = dados.dropna()

dados = dados[["trades", "rentabilidade_trades"]]

dados

# Passo 9 - Calcular estatísticas do nosso modelo de investimento (EV, Trades certos, Rentabilidade acumulada)

In [ ]:
#Estatisticas

rentabilidade_acumulada = ((1 + dados["rentabilidade_trades"]).cumprod() - 1).iloc[-1]
trades_certos = (len(dados[dados["rentabilidade_trades"] > 0]))/len(dados["rentabilidade_trades"])
trades_errados = 1 - trades_certos
media_ganhos = dados[dados["rentabilidade_trades"] > 0]["rentabilidade_trades"].mean()
media_perdas = dados[dados["rentabilidade_trades"] <= 0]["rentabilidade_trades"].mean()
EV = (trades_certos * media_ganhos) - (trades_errados * abs(media_perdas))

print(f'''
Rentabilidade Acumulada: {rentabilidade_acumulada}
Trades Certos: {trades_certos}
EV: {EV}


''')

In [ ]:
scatter = dados["rentabilidade_trades"].to_frame()
scatter = scatter.reset_index()
scatter.columns = ["Data", "Rentabilidade do Trade"]
ax = scatter.plot(x = "Data", y = "Rentabilidade do Trade", kind = "scatter")
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
ax.axhline(y=0, color = "red", linestyle='--')